In [1]:
import sympy
from phasor.utilities.ipynb.displays import *
from phasor.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from phasor.system import DAG_algorithm
from phasor.system import SRE_matrix_algorithms
from phasor.system import scisparse_algorithm

import timeit

asavefig.org_subfolder = 'plots'

Populating the interactive namespace from numpy and matplotlib
Sympy version:  1.0


In [48]:
from functools import reduce
def SVD_genmatrix(
    N = 10,
    length = 1,
):
    U = gen_rand_unitary(N = N, length = length)
    V = gen_rand_unitary(N = N, length = length)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)
    sparsity = SRE_matrix_algorithms.SRE_count_sparsity(M)
    edge_map = M[2]
    
    m = scisparse.dok_matrix((N, N), dtype = complex)
    for (k1, k2), e in edge_map.items():
        m[k1, k2] = e
    mtrx = scisparse.csc_matrix(m)
    
    b = declarative.Bunch( 
        edge_map = edge_map,
        csc = mtrx,
    )
    b.update(sparsity)
    return b

In [49]:
mat = SVD_genmatrix(N = 300)
import scipy.sparse as scisparse

<300x300 sparse matrix of type '<class 'numpy.complex128'>'
	with 974 stored elements in Compressed Sparse Column format>

In [41]:
#imshow(abs(mtrx.toarray()) != 0)
import scipy.io

In [50]:
scipy.io.savemat(
    'mat_tests.mat', 
     mdict=dict(
         test_100 = SVD_genmatrix(N = 100).csc,
         test_300 = SVD_genmatrix(N = 300).csc,
         test_1000 = SVD_genmatrix(N = 1000).csc,
     ),
)